In [2]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.compute as pc
import urllib.parse
import pyarrow

In [3]:
os.environ["AWS_SHARED_CREDENTIALS_FILE"] = "../../srv/keys/credentials"

## 1. Setup your EC2 instance
![img](img/milestone_2-step_1.png)

## 2. Setup the server
![img](img/milestone_2-step_2.png)

## 3. Setup your JupyterHub
![img](img/milestone_2-step_3.png)

## 4. Get the data what we wrangled in our first milestone.

In [ ]:
article_id = 14226968
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../../srv/data/group_4_shared_folder/"

In [ ]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]           
files

In [ ]:
files_to_dl = ["combined_model_data_parti.parquet.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [ ]:
with zipfile.ZipFile(os.path.join(output_directory, "combined_model_data_parti.parquet.zip"), 'r') as f:
    f.extractall(output_directory)

## 5. Setup your S3 bucket and move data
![img](img/milestone_2-step_4.png)

In [2]:
# !aws s3 cp combined_model_data_parti.parquet/ s3://mds-s3-4-mrnabiz/combined_model_data_parti.parquet --recursive

## 6. Wrangle the data in preparation for machine learning

In [4]:
df_syd = pd.read_csv("s3://mds-s3-4-mrnabiz/observed_daily_rainfall_SYD.csv")
df_syd.head(10)

,time,rain (mm/day)
0,1889-01-01,0.006612
1,1889-01-02,0.090422
2,1889-01-03,1.401452
3,1889-01-04,14.869798
4,1889-01-05,0.467628
5,1889-01-06,0.000000
6,1889-01-07,0.403645
7,1889-01-08,0.111741
8,1889-01-09,0.000000
9,1889-01-10,0.000000


In [5]:
table = pq.read_table("s3://mds-s3-4-mrnabiz/combined_model_data_parti.parquet")
table

pyarrow.Table
time: timestamp[ms]
lat_min: double
lat_max: double
lon_min: double
lon_max: double
rain (mm/day): double
model: dictionary<values=string, indices=int32, ordered=0>
----
time: [[1889-01-01 12:00:00.000,1889-01-02 12:00:00.000,1889-01-03 12:00:00.000,1889-01-04 12:00:00.000,1889-01-05 12:00:00.000,...,1995-11-09 12:00:00.000,1995-11-10 12:00:00.000,1995-11-11 12:00:00.000,1995-11-12 12:00:00.000,1995-11-13 12:00:00.000],[1995-11-14 12:00:00.000,1995-11-15 12:00:00.000,1995-11-16 12:00:00.000,1995-11-17 12:00:00.000,1995-11-18 12:00:00.000,...,1976-09-21 12:00:00.000,1976-09-22 12:00:00.000,1976-09-23 12:00:00.000,1976-09-24 12:00:00.000,1976-09-25 12:00:00.000],...,[2008-09-27 12:00:00.000,2008-09-28 12:00:00.000,2008-09-29 12:00:00.000,2008-09-30 12:00:00.000,2008-10-01 12:00:00.000,...,2014-12-27 12:00:00.000,2014-12-28 12:00:00.000,2014-12-29 12:00:00.000,2014-12-30 12:00:00.000,2014-12-31 12:00:00.000],[1889-01-01 00:00:00.000,1889-01-02 00:00:00.000,1889-01-03 00:00:0

In [6]:
column1_name = 'lat_min'
column2_name = 'lat_max'
column3_name = 'lon_min'
column4_name = 'lon_max'

syd_lat = -33.86
syd_lon = 151.21

mask = pc.and_(
    pc.and_(
        pc.greater_equal(table[column2_name], syd_lat),
        pc.less_equal(table[column1_name], syd_lat)),
    pc.and_(
        pc.greater_equal(table[column4_name], syd_lon),
        pc.less_equal(table[column3_name], syd_lon)))
        
filtered_table = table.filter(mask)
df = filtered_table.to_pandas()
df.head(10)

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.0,-33.75,150.0,151.875,0.040427,ACCESS-CM2
1,1889-01-02 12:00:00,-35.0,-33.75,150.0,151.875,0.073777,ACCESS-CM2
2,1889-01-03 12:00:00,-35.0,-33.75,150.0,151.875,0.232656,ACCESS-CM2
3,1889-01-04 12:00:00,-35.0,-33.75,150.0,151.875,0.911319,ACCESS-CM2
4,1889-01-05 12:00:00,-35.0,-33.75,150.0,151.875,0.698013,ACCESS-CM2
5,1889-01-06 12:00:00,-35.0,-33.75,150.0,151.875,0.176846,ACCESS-CM2
6,1889-01-07 12:00:00,-35.0,-33.75,150.0,151.875,0.347102,ACCESS-CM2
7,1889-01-08 12:00:00,-35.0,-33.75,150.0,151.875,0.042755,ACCESS-CM2
8,1889-01-09 12:00:00,-35.0,-33.75,150.0,151.875,0.029409,ACCESS-CM2
9,1889-01-10 12:00:00,-35.0,-33.75,150.0,151.875,0.023641,ACCESS-CM2


In [7]:
pivot_df = df.drop(columns=['lat_min', 
                            'lat_max', 
                            'lon_min', 
                            'lon_max'])

pivot_df['time'] = pivot_df['time'].dt.date

pivot_df = pivot_df.pivot(index='time',
                          columns='model', 
                          values='rain (mm/day)')
pivot_df.head(10)

model,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MIROC6,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,35.579336,4.268112e+00,1.107466e-03,11.410537,3.322009e-08,2.668800,1.321215,1.515293e+00,...,0.648031,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933
1889-01-02,0.073777,0.303965,4.596520,1.190141e+00,1.015323e-04,4.014984,1.312700e+00,0.946211,2.788724,4.771375e+00,...,1.099883,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381
1889-01-03,0.232656,0.019976,5.927467,1.003845e-09,1.760345e-05,9.660565,9.103720e+00,0.431999,0.003672,4.233980e+00,...,3.931751,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909
1889-01-04,0.911319,13.623777,8.029624,8.225225e-02,1.808932e-01,3.951528,1.317160e+01,0.368693,0.013578,1.525249e+01,...,0.627365,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472226e-07,5.301038e-01,0.002699,2.185454,2.106737
1889-01-05,0.698013,0.021048,2.132686,2.496841e+00,4.708019e-09,2.766362,1.822940e+01,0.339267,0.002468,1.192036e+01,...,0.019777,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335
1889-01-06,0.176846,0.364671,0.106105,4.383307e-13,3.437249e-07,23.617552,4.036332e+00,0.008937,0.000000,7.534027e-02,...,0.490276,3.888130e-01,1.212115e-13,4.027802e+00,8.058601e-01,1.926850e-13,1.192076e-03,2.155050,3.281224,0.096504
1889-01-07,0.347102,0.012561,0.780772,4.599509e-04,1.598180e+01,9.254538,5.343101e+00,15.105793,0.211540,3.154230e+00,...,35.833649,2.101678e-02,1.187939e-13,2.491990e-01,8.221049e-13,1.490119e-01,6.488937e-01,6.970733,1.117913,3.021798
1889-01-08,0.042755,0.087271,0.312471,1.182123e-06,8.529780e+00,17.310366,7.936454e+00,36.009891,0.013135,2.109385e+00,...,2.013104,2.094235e-01,1.931241e-01,9.888659e-14,1.766779e-03,8.325429e+00,4.708920e-01,0.858247,53.212309,2.886699
1889-01-09,0.029409,0.367011,0.698855,1.814032e+00,1.793289e-01,21.376961,2.659956e+00,2.105408,23.622214,-3.911231e-20,...,0.028268,4.433096e-13,2.564862e-03,4.506111e+00,5.282789e+00,1.833132e-13,4.403384e-03,3.504712,0.583404,2.902378


In [8]:
df_syd = df_syd.rename(columns = {'rain (mm/day)':'Observed',
                                  'time':'time-o'})
df_syd['time-o'] = pd.to_datetime(df_syd['time-o'])
df_syd['time-o'] = df_syd['time-o'].dt.date
pivot_df_ind = pivot_df.reset_index()
concatenated_df = pd.concat([pivot_df_ind, df_syd], axis=1)
concatenated_df = concatenated_df.drop(columns = ['time-o']).set_index('time')
concatenated_df.head(10)

,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1,Observed
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,35.579336,4.268112e+00,1.107466e-03,11.410537,3.322009e-08,2.668800,1.321215,1.515293e+00,...,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933,0.006612
1889-01-02,0.073777,0.303965,4.596520,1.190141e+00,1.015323e-04,4.014984,1.312700e+00,0.946211,2.788724,4.771375e+00,...,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381,0.090422
1889-01-03,0.232656,0.019976,5.927467,1.003845e-09,1.760345e-05,9.660565,9.103720e+00,0.431999,0.003672,4.233980e+00,...,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909,1.401452
1889-01-04,0.911319,13.623777,8.029624,8.225225e-02,1.808932e-01,3.951528,1.317160e+01,0.368693,0.013578,1.525249e+01,...,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472226e-07,5.301038e-01,0.002699,2.185454,2.106737,14.869798
1889-01-05,0.698013,0.021048,2.132686,2.496841e+00,4.708019e-09,2.766362,1.822940e+01,0.339267,0.002468,1.192036e+01,...,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335,0.467628
1889-01-06,0.176846,0.364671,0.106105,4.383307e-13,3.437249e-07,23.617552,4.036332e+00,0.008937,0.000000,7.534027e-02,...,3.888130e-01,1.212115e-13,4.027802e+00,8.058601e-01,1.926850e-13,1.192076e-03,2.155050,3.281224,0.096504,0.000000
1889-01-07,0.347102,0.012561,0.780772,4.599509e-04,1.598180e+01,9.254538,5.343101e+00,15.105793,0.211540,3.154230e+00,...,2.101678e-02,1.187939e-13,2.491990e-01,8.221049e-13,1.490119e-01,6.488937e-01,6.970733,1.117913,3.021798,0.403645
1889-01-08,0.042755,0.087271,0.312471,1.182123e-06,8.529780e+00,17.310366,7.936454e+00,36.009891,0.013135,2.109385e+00,...,2.094235e-01,1.931241e-01,9.888659e-14,1.766779e-03,8.325429e+00,4.708920e-01,0.858247,53.212309,2.886699,0.111741
1889-01-09,0.029409,0.367011,0.698855,1.814032e+00,1.793289e-01,21.376961,2.659956e+00,2.105408,23.622214,-3.911231e-20,...,4.433096e-13,2.564862e-03,4.506111e+00,5.282789e+00,1.833132e-13,4.403384e-03,3.504712,0.583404,2.902378,0.000000


In [9]:
concatenated_df.to_csv('../../srv/data/group_4_shared_folder/ml_data_SYD.csv')

In [10]:
# !aws s3 cp ml_data_SYD.csv s3://mds-s3-4-mrnabiz/output/ml_data_SYD.csv